# Sandbox env. for developing middleware classes

A generic dataclass representing a bet on an event

In [2]:
from datetime import datetime
from typing import Set
from pydantic import BaseModel

class Bet(BaseModel):
    """
    Represents a betting event.

    Attributes:
        event_id (int): The unique identifier of the event.
        bookmaker_id (int): The unique identifier of the bookmaker.
        sport (str): The sport of the betting event.
        event_date (datetime): The date and time of the event.
        event_date (datetime): The timezone in which the event occurs.
        participants (set): The set of participants in the event.
        outcome (str): The selected outcome of the bet for the target participant.
        target (int): The participant to which the chosen outcome refers.
        stake (float): The amount wagered on the bet.
        odds (float): The odds of the bet.
        odds_unit (str): The selected odds standard, ie. fractional, american, or decimal.
    """

    event_id: int
    bookmaker_id: int
    sport: str
    event_date: datetime
    event_tz: str
    participants: Set[str]
    outcome: str
    target: str
    stake: float
    odds: float
    odds_unit: str


    @classmethod
    def create(cls, **data):
        """
        Custom class method to create a Bet object.

        Args:
            **data: Keyword arguments representing the attributes of the Bet object.

        Returns:
            Bet: The created Bet object.

        Raises:
            ValueError: If the target participant is not in the participants set.
            ValueError: If the event_tz string is not a valid timezone.
        """
        instance = cls(**data)
        
        # Additional validation
        print(f'post init checks - target: {instance.target}, participants: {instance.participants}')
        if instance.target not in instance.participants:
            raise ValueError("Target participant must be one of the participants")
        
        # You can add more checks for other attributes here
        
        # Check if event_tz is a valid timezone
        from pytz import all_timezones
        if instance.event_tz not in all_timezones:
            raise ValueError("Invalid timezone")
        
        return instance



## Validating class attributes

The below cell is how we expect the class to be created

In [3]:
bet = Bet.create(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Perth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Silly Goose',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal'
)

print(bet)

post init checks - target: Silly Goose, participants: {'Daffy Duck', 'Silly Goose'}
event_id=1 bookmaker_id=1 sport='duck duck goose' event_date=datetime.datetime(2021, 6, 1, 12, 0) event_tz='Australia/Perth' participants={'Daffy Duck', 'Silly Goose'} outcome='win' target='Silly Goose' stake=99.0 odds=1.4 odds_unit='decimal'


Here we test the type checking from `pydantic`

In [4]:
my_instance = Bet.create(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Perth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Silly Goose',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal')
print(my_instance) # Output: MyClass attribute1=10 attribute2='Hello'


# Incorrect usage - raises ValidationError
try:
    my_instance = Bet.create(
        event_id = 1,
        bookmaker_id = 'invalid',
        sport = 'duck duck goose',
        event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
        event_tz = "Australia/Perth",
        participants = {'Silly Goose', 'Daffy Duck'},
        outcome = 'win',
        target = 'Silly Goose',
        stake = 99.0,
        odds = 1.4,
        odds_unit = 'decimal'
        )
    print(my_instance)
except Exception as e:
    print(str(e))

post init checks - target: Silly Goose, participants: {'Daffy Duck', 'Silly Goose'}
event_id=1 bookmaker_id=1 sport='duck duck goose' event_date=datetime.datetime(2021, 6, 1, 12, 0) event_tz='Australia/Perth' participants={'Daffy Duck', 'Silly Goose'} outcome='win' target='Silly Goose' stake=99.0 odds=1.4 odds_unit='decimal'
1 validation error for Bet
bookmaker_id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='invalid', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/int_parsing


Here we test out `__post_init__` method validations

In [5]:
try:
    my_instance = Bet.create(
        event_id = 1,
        bookmaker_id = 1,
        sport = 'duck duck goose',
        event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
        event_tz = "Australia/Perth",
        participants = {'Silly Goose', 'Daffy Duck'},
        outcome = 'win',
        target = 'Stupid Bird',
        stake = 99.0,
        odds = 1.4,
        odds_unit = 'decimal')
    print(my_instance)
except ValueError as e:
    print(str(e))

post init checks - target: Stupid Bird, participants: {'Daffy Duck', 'Silly Goose'}
Target participant must be one of the participants


In [7]:
# Incorrect usage - raises ValidationError
try:
    my_instance = Bet.create(
        event_id = 1,
        bookmaker_id = 1,
        sport = 'duck duck goose',
        event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
        event_tz = "Australia/Purth",
        participants = {'Silly Goose', 'Daffy Duck'},
        outcome = 'win',
        target = 'Silly Goose',
        stake = 99.0,
        odds = 1.4,
        odds_unit = 'decimal'
        )
    print(my_instance)
except Exception as e:
    print(str(e))

post init checks - target: Silly Goose, participants: {'Daffy Duck', 'Silly Goose'}
Invalid timezone


Let's restrict direct instantiation of the class so that we **must** use the create method to create an instance of the class. We'll create a singleton class that inherits from BaseModel, and then a custom base model that inherits from both of these classes, our Bet class will then inherit from this CustomBaseModel

In [8]:
from datetime import datetime
from typing import Set
from pydantic import BaseModel



class NoDirectInstantiationMeta(type):
    """
    A metaclass for preventing direct instantiation of classes.
    """
    def __call__(cls, *args, **kwargs):
        raise ValueError("This class cannot be instantiated directly. Use the create() method instead.")
    

class CustomBaseModel(BaseModel, metaclass=Singleton):
    pass


class Bet(CustomBaseModel):
    """
    Represents a betting event.

    Attributes:
        event_id (int): The unique identifier of the event.
        bookmaker_id (int): The unique identifier of the bookmaker.
        sport (str): The sport of the betting event.
        event_date (datetime): The date and time of the event.
        event_date (datetime): The timezone in which the event occurs.
        participants (set): The set of participants in the event.
        outcome (str): The selected outcome of the bet for the target participant.
        target (int): The participant to which the chosen outcome refers.
        stake (float): The amount wagered on the bet.
        odds (float): The odds of the bet.
        odds_unit (str): The selected odds standard, i.e., fractional, american, or decimal.
    """

    event_id: int
    bookmaker_id: int
    sport: str
    event_date: datetime
    event_tz: str
    participants: Set[str]
    outcome: str
    target: str
    stake: float
    odds: float
    odds_unit: str

    @classmethod
    def create(cls, **data):
        """
        Custom class method to create a Bet object.

        Args:
            **data: Keyword arguments representing the attributes of the Bet object.

        Returns:
            Bet: The created Bet object.

        Raises:
            ValueError: If the target participant is not in the participants set.
            ValueError: If the event_tz string is not a valid timezone.
        """
        instance = cls(**data)

        print(f'post init checks - target: {instance.target}, participants: {instance.participants}')
        if instance.target not in instance.participants:
            raise ValueError("Target participant must be one of the participants")

        # You can add more checks for other attributes here

        # Check if event_tz is a valid timezone
        from pytz import all_timezones
        if instance.event_tz not in all_timezones:
            raise ValueError("Invalid timezone")

        return instance


new_instance = Bet.create(
    event_id=1,
    bookmaker_id=1,
    sport='duck duck goose',
    event_date=datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz="Australia/Perth",
    participants={'Silly Goose', 'Daffy Duck'},
    outcome='win',
    target='Silly Goose',
    stake=99.0,
    odds=1.4,
    odds_unit='decimal'
)
print(my_instance)

# Attempting to instantiate directly will raise an error
try:
    my_instance_direct = Bet(
        event_id=2,
        bookmaker_id=2,
        sport='duck duck goose',
        event_date=datetime.now(),
        event_tz="Australia/Perth",
        participants={'Silly Goose', 'Daffy Duck'},
        outcome='win',
        target='Silly Goose',
        stake=99.0,
        odds=1.4,
        odds_unit='decimal'
    )
except ValueError as e:
    print(str(e))


ValueError: This class cannot be instantiated directly. Use the create() method instead.

Now if I try to create another instance using the create method

In [9]:
new_instance = Bet.create(
    event_id=1,
    bookmaker_id=1,
    sport='duck duck goose',
    event_date=datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz="Australia/Perth",
    participants={'Silly Goose', 'Daffy Duck'},
    outcome='win',
    target='Silly Goose',
    stake=99.0,
    odds=1.4,
    odds_unit='decimal'
)
print(my_instance)

ValueError: This class cannot be instantiated directly. Use the create() method instead.

In [ ]:
import pytz

for tz in pytz.all_timezones:
    print(tz)


In [ ]:
import pytz

# Create a string representing the datetime
datetime_str = '2021-06-01 12:00:00'

# Parse the string to a naive datetime object
naive_datetime = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')

# Create a timezone-aware datetime object
timezone = pytz.timezone('Australia/Melbourne')
aware_datetime = timezone.localize(naive_datetime)

# Print the timezone-aware datetime object
print(aware_datetime)


In [ ]:
Bet(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Perth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Silly Goose',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal',
)

Test validations

In [ ]:
Bet(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Perth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Stupid bird',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal',
)

In [ ]:
Bet(
    event_id = 1,
    bookmaker_id = 1,
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Parth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Silly Goose',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal',
)

In [ ]:
Bet(
    event_id = 1,
    bookmaker_id = '1',
    sport = 'duck duck goose',
    event_date = datetime.strptime('2021-06-01 12:00:00', '%Y-%m-%d %H:%M:%S'),
    event_tz = "Australia/Perth",
    participants = {'Silly Goose', 'Daffy Duck'},
    outcome = 'win',
    target = 'Silly Goose',
    stake = 99.0,
    odds = 1.4,
    odds_unit = 'decimal',
)

In [ ]:
from pydantic import BaseModel

class MyClass(BaseModel):
    attribute1: int
    attribute2: str

my_instance = MyClass(attribute1=10, attribute2="Hello")
print(my_instance) # Output: MyClass attribute1=10 attribute2='Hello'

# Incorrect usage - raises ValidationError
my_instance = MyClass(attribute1="invalid", attribute2=20)
print(my_instance)


In [ ]:
my_instance = MyClass(10, 20)
